In [44]:
# %%
import pandas as pd
from collections import namedtuple

def check_column_(df, column_name, total_number):
    x = df[column_name]
    x = x[~pd.isnull(x)]
    return len(x) == total_number & len(set(x)) == total_number

classe = namedtuple('classe', ['teacher', 'day_hour'])

def duplicated_indices_in_column(df, column_name):
    x = df[column_name]
    duplicated_indices = x[~pd.isnull(x)].index[x[~pd.isnull(x)].duplicated(keep=False)]
    return df.loc[duplicated_indices, column_name]

# style_this is a set of tuples (index, column)
def dump_to_html(df, style_this):
    html_string = '<table style="width:80%">'
    header = "<tr>"
    for col in df.columns:
        header += f"<th>{col}</th>"
    header += "</th></tr>"
    html_string += header
    for _, row in df.iterrows():
        html_string += "<tr>"
        index = row['teacher']
        for col in df.columns:
            if (index, col) in style_this:
                html_string += f'<td style="background-color:Orange;">{row[col]}</td>' if pd.notnull(row[col]) else "<td></td>"  
            else:
                html_string += f"<td>{row[col]}</td>" if pd.notnull(row[col]) else "<td></td>"  
        html_string += "</tr>"
    html_string += "</table>"
    return html_string

def wite_to_html_file(html, filename):
    with open(filename, "w") as f:
        f.write(html)

def permute(df, classe_1, classe_2 ):
    classroom_temp = df.loc[classe_1.teacher, classe_1.day_hour] 
    df.loc[classe_1.teacher, classe_1.day_hour] = df.loc[classe_2.teacher, classe_2.day_hour]
    df.loc[classe_2.teacher, classe_2.day_hour] = classroom_temp
    # Check if there are duplicates
    first_column_name = classe_1.day_hour
    second_column_name = classe_2.day_hour
    first_column_duplicated_indices = duplicated_indices_in_column(df, first_column_name)
    second_column_duplicated_indices = duplicated_indices_in_column(df, second_column_name)
    return (first_column_duplicated_indices, first_column_name), (second_column_duplicated_indices, second_column_name)

In [41]:
to_style = [
    (ph.teacher, ph.day + str(ph.hour)) for ph in permute_history for ph in ph
]

In [35]:
def find_next_available_slot(df, teacher):
    for col in df.columns:
            val = df.loc[teacher, col]
            if val=='' or pd.isnull(val):
                return col
    return None


orario = pd.read_csv('./data/to_v1.csv')
orario.set_index("teacher", inplace=True)
permute_history = []

conflicting_teacher = 'ASTORNAIUOLO L.'
conflicting_hour_1 = 'GIO11'
conflicting_hour_2 = 'GIO14'
c1 = classe(teacher=conflicting_teacher, day_hour=conflicting_hour_1)
c2 = classe(teacher=conflicting_teacher, day_hour=conflicting_hour_2)
conflict1, conflict2 = permute(orario, c1, c2)
permute_history.append( (c1, c2) )

conflicts = conflict1[0].size != 0 or conflict2[0].size != 0

max_item = 20
iterations = 0
while conflicts and iterations < max_item:
    # find which teacher different from the current one has conflicts
    if conflict1[0].size != 0:
        conflicting_teacher = list(set(conflict1[0]) - set([conflicting_teacher]))[0]
        conflicting_class = conflict1[1]
    elif conflict2[0].size != 0:
        conflicting_teacher = list(set(conflict2[0]) - set([conflicting_teacher]))[0]
        conflicting_class = conflict2[1]
    c1 = classe(teacher=conflicting_teacher, day_hour=conflicting_class)
    free_day_hour = find_next_available_slot(orario, conflicting_teacher)
    c2 = classe(teacher=conflicting_teacher, day_hour=free_day_hour)
    conflict1, conflict2 = permute(orario, c1, c2)
    permute_history.append( (c1, c2) )
    if conflict1[0].size == 0 and conflict2[0].size == 0:
        print("No more conflicts!")
        conflicts = False
    iterations += 1

In [45]:
string = dump_to_html(orario, to_style)
wite_to_html_file(string, "orario_permutato.html")

In [ ]:
def holes_by_day(df, teacher):
    """
    Returns a dictionary {day: number_of_holes} for the given teacher.
    Assumes columns are like 'LUN8', 'LUN9', ..., 'VEN14'.
    A hole is a gap (empty slot) between two non-empty slots in the same day.
    """
    days = ['LUN', 'MAR', 'MER', 'GIO', 'VEN']
    hours = [8, 9, 10, 11, 12, 13, 14]
    holes = {}
    for day in days:
        # Get the values for the teacher for this day, in hour order
        vals = [df.loc[teacher, f"{day}{hour}"] if f"{day}{hour}" in df.columns else None for hour in hours]
        # Find indices of non-empty slots
        non_empty = [i for i, v in enumerate(vals) if pd.notnull(v) and v != '']
        if len(non_empty) <= 1:
            holes[day] = 0
            continue
        # Count the number of empty slots between first and last non-empty
        first, last = non_empty[0], non_empty[-1]
        n_holes = 0
        for i in range(first, last):
            if i not in non_empty:
                n_holes += 1
        holes[day] = n_holes
    return holes

orario = pd.read_csv('./data/to_v1.csv')
orario.set_index("teacher", inplace=True)
LUN14 = ['3E', '4E', '5E', '1L', '1I', '2L', '2I', '3L']
docentes = pd.read_csv('./data/docente_classes.csv')
import numpy as np
total_classes = set(docentes['Classi']) - set([np.nan])
not_allowed = {
    'LUN14': total_classes - set(LUN14),
    'MER14': total_classes - set(LUN14), 
    'VEN14': total_classes - set(['3L'])
}

def allowed_permutation(df, classe_1, classe_2):
        # check if possible if they where to change places
    classroom_1 = df.loc[classe_1.teacher, classe_1.day_hour]
    classroom_2 = df.loc[classe_2.teacher, classe_2.day_hour]
    classroom_1_allowed = True
    classroom_2_allowed = True
    
    classroom_1_allowed = classroom_1 not in not_allowed.get(classe_2.day_hour, set())
    classroom_2_allowed = classroom_2 not in not_allowed.get(classe_1.day_hour, set())
    if not classroom_1_allowed or not classroom_2_allowed:
        return False
    
    


In [103]:
orario = pd.read_csv('solucion_v1.csv')
orario.set_index('teacher', inplace=True)
holes = holes_by_day(orario, "STORNAIUOLO L.")
print(holes)

{'LUN': 3, 'MAR': 1, 'MER': 0, 'GIO': 0, 'VEN': 1}
